# Programmieraufgabe 07: Lokales und Globales Newton-Verfahren

In dieser Aufgabe wollen wir das lokale und globale Newton-Verfahren aus der Vorlesung implementieren und die Abhängigkeit der benötigten Iterationsschritte vom Startwert untersuchen.


Tragen Sie zunächst in der folgenden Zelle **beide** Ihre Namen ein: 

In [ ]:
# Numerische Optimierungsverfahren der Wirtschaftsmathematik
# Wintersemester 2025/2026
# Übungsblatt 6 - Programmieraufgabe 6
#
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]
# 
# [Nachname], [Vorname]
# [Vorname.Nachname@uni-a.de]

In [ ]:
import math
import numpy as np
import numpy.linalg as la

# Diese Zeilen werden für die plots benötigt
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

Implementieren Sie zunächst das Lokale Newton-Verfahren (Algorithmus 3.8). Das Verfahren soll abbrechen, wenn entweder die maximale Anzahl an Iterationen `kmax` durchgeführt wurde, oder der Abstand der Funktionswerte zweier Iterationen $|f(x^{(k)}) - f(x^{(k-1)})|$ kleiner als die Toleranz `tol` ist.

Für das Lösen der Newton-Gleichung dürfen sie dabei die Funktion `la.solve` verwenden.

In [ ]:
def newton_local(f, g, h, x_0, tol, kmax):
    '''
        Lokales Newton-Verfahren

        Parameter:
            f        : Die zu minimierende Funktion f
            g        : Funktion, die den Gradienten von f zurückgibt
            h        : Funktion, die die Hesse-Matrix von f zurückgibt
            x_0      : Startvektor
            tol      : Abbruchkriterium
            kmax     : Maximale Anzahl an Iterationen

        Rückgabewert:
            iterates : Vektor der Iterierten. Dabei ist iterates[-1,:]
                       der Rückgabewert und len(iterates) - 1 die Anzahl
                       an Iterationen
    '''
    
    #  Initialisierung
    n = x_0.shape[0]
    iterates = np.ones((kmax + 1, n))
    x = np.copy(x_0)
    iterates[0,:] = x
    f_old = f(x)
    f_diff = np.inf

    ???
    
    return iterates

Sie können die Folgende Zelle zum Testen Ihrer Funktion verwenden. Dabei sollte das lokale Newton-Verfahren in zwei Schritten zum Minimum (0, 0) konvergieren (Hier wird ein Schritt benötigt, um zum Minimum zu gelangen, und ein weiterer, um festzustellen, dass sich der Funktionswert nicht mehr ändert).

In [ ]:
f = lambda x : x[0]**2 + x[0]**2
g = lambda x : np.array([2*x[0], 2*x[1]])
h = lambda x : np.array([[2, 0], [0, 2]])
iterates = newton_local(f, g, h, np.array([1.5, -99]), 1e-16, 100)
print(iterates)
print(f'newton_local konvergiert in {len(iterates)-1} Schritten zu {iterates[-1]}.')

Implementieren Sie nun die Armijo-Schrittweitenregel für das globale Newton-Verfahren. Sie können wieder Ihre Lösung aus Aufgabe 3 größtenteils wiederverwenden, achten Sie aber darauf, dass die Funktion `f` nicht unbedingt quadratisch sein muss.

In [ ]:
def armijo_step_size(f, g, x, direction, beta = 0.5, gamma = 1e-4):
    '''
        Armijo-Schrittweitenregel

        Parameter:
            f           : Die zu minimierende Funktion f
            g           : Der Gradient von f
            x           : Aktueller Wert der Iteration
            direction   : Aktuelle Richtung des Abstiegsverfahrens
            beta, gamma : Parameter der Armijo-Schritweitenregel

        Rückgabewert:
            alpha: Armijo-Schrittweite
    '''
    
    if beta <= 0 or beta >= 1:
        raise ValueError(f'Der Parameter beta muss im Intervall (0,1) liegen, aber beta={beta}!')
    if gamma <= 0 or gamma >= 1:
        raise ValueError(f'Der Parameter gamma muss im Intervall (0,1) liegen, aber gamma={gamma}!')

   ???

    return alpha

Mit der folgenden Zelle können Sie die Schrittweite testen. Dabei sollte das Ergebnis der ersten Auswerktung `0.5625` und der zweiten `0.75` sein.

Die dritte Auswertung testet den Fall, dass `direction` keine Abstiegsrichtung ist. Dies kann im Newton-Verfahren leicht auftreten, z.B. wenn das exakte Minimum erreicht wird. Geben Sie in diesem Fall `alpha = 1` zurück und vermeiden Sie Endlosschleifen!

In [ ]:
f = lambda x : np.array([math.sin(x[0] * math.pi)])
g = lambda x : np.array([math.cos(x[0] * math.pi)])
print(armijo_step_size(f, g, np.array([0.0]), np.array([-1.0]), beta = 0.75, gamma = 0.99))
print(armijo_step_size(f, g, np.array([0.0]), np.array([-1.0]), beta = 0.75, gamma = 0.1))
print(armijo_step_size(f, g, np.array([0.0]), np.array([ 0.0]), beta = 0.75, gamma = 0.1))

Als letztes implementieren Sie das globale Newton-Verfahren (Algorithmus 3.9).

Wie im lokalen Newton-Verfahren dürfen Sie `la.solve` zum Lösen der Newtongleichung verwenden. Um zu testen, ob die Hesse-Matrix numerisch nicht invertierbar ist, verwenden Sie z.B. `abs(la.cond(hessian)) < 1e-14`.

In [ ]:
def newton_global(f, g, h, x_0, tol, kmax, step_size, delta = 0.01, p = 3):
    '''
        Globales Newton-Verfahren

        Parameter:
            f         : Die zu minimierende Funktion f
            g         : Funktion, die den Gradienten von f zurückgibt
            h         : Funktion, die die Hesse-Matrix von f zurückgibt
            x_0       : Startvektor
            tol       : Abbruchkriterium
            kmax      : Maximale Anzahl an Iterationen
            step_size : Funktion, die als Argumente f, g, die aktuelle 
                        Iterierte x und die aktuelle Richtung direction 
                        erhält und eine Schrittweite alpha zurückgibt
            delta, p  : Parameter des Globalen Newton-Verfahrens

        Rückgabewert:
            iterates : Vektor der Iterierten. Dabei ist iterates[-1,:]
                       der Rückgabewert und len(iterates) - 1 die Anzahl
                       an Iterationen
    '''

    #  Initialisierung
    n = x_0.shape[0]
    iterates = np.ones((kmax + 1, n))
    x = np.copy(x_0)
    iterates[0,:] = x
    k = 1
    f_old = f(x)
    f_diff = np.inf

    ???
    
    return iterates

Testen Sie wieder Ihre Implementierung mit der folgenden Zelle. Auch `newton_global` sollte in zwei Schritten zu (0, 0) konvergieren (Wie oben wird der zweite Schritt nur benötigt, um das Abbruchkriterium zu verifizieren).

In [ ]:
f = lambda x : x[0]**2 + x[0]**2
g = lambda x : np.array([2*x[0], 2*x[1]])
h = lambda x : np.array([[2, 0], [0, 2]])
iterates = newton_global(f, g, h, np.array([1.5, -99]), 1e-16, 100, armijo_step_size)
print(iterates)
print(f'newton_local konvergiert in {len(iterates)-1} Schritten zu {iterates[-1]}.')

Im Folgenden definieren wir uns drei Testfunktionen: Die Rosenbrock-Funktion $f_R(x_1, x_2) = 100(x_2 - x_1^2)^2 + (1 - x_1)^2$ aus Beispiel 3.48, die Himmelblau-Funktion $f_H(x_1, x_2) = (x_1^2 + x_2 - 11)^2 + (x_1 + x_2^2 - 7)^2$, die ebenfalls eine beliebte Testfunktion für Optimierungsalgorithmen ist, und die Funktion $f_S(x_1, x_2) = x_1 (x_1^2 - 2 x_2^2) + 0.15(x_1^2 + x_2^2 - 1)^2$.

In [ ]:
def  rosenbrock_funktion(x):
    return 100.0 *(x[1] - x[0]**2)**2 + (1.0 - x[0])**2

def  rosenbrock_gradient ( x ) :
    return np.array(
        [
            400.0 * x[0] * (x[0]**2 - x[1]) + 2.0 * (x[0] - 1.0),
            200.0 * (x[1] - x[0]**2),
        ],
    )

def  rosenbrock_hesse(x):
    return np.array(
        [
            [
                1200.0 * x[0]**2 - 400.0 * x[1] + 2.0, 
                -400.0 * x[0],
            ],
            [
                -400.0 * x[0],
                200.0,
            ],
        ],
    )
rosenbrock = (rosenbrock_funktion, rosenbrock_gradient, rosenbrock_hesse)

In [ ]:
def himmelblau_funktion(x):
    return (x[0]**2 + x[1] - 11.0 )**2 + (x[0] + x[1]**2 - 7.0)**2

def  himmelblau_gradient(x):
    return np.array(
        [
            4.0 * x[0] * (x[0]**2 + x[1] - 10.5) + 2.0 * x[1]**2 - 14.0, 
            4.0 * x[1] * (x[1]**2 + x[0] - 6.5 ) + 2.0 * x[0]**2 - 22.0,
        ],
    )
    
def  himmelblau_hesse(x):
    return np.array(
        [
            [
                12.0 * x[0]**2 + 4.0 * x[1] - 42.0,
                4.0 * (x[0] + x[1]),
            ],
            [
                4.0 * (x[0] + x[1]),
                12.0 * x[1]**2 + 4.0 * x[0] - 26.0,
            ],
        ],
    )
himmelblau = (himmelblau_funktion, himmelblau_gradient, himmelblau_hesse)

In [ ]:
def sattel_funktion(x):
    return x[0] * (x[0]**2 - 2.0 * x[1]**2) + 0.15 * (x[0]**2 + x[1]**2 - 1.0)**2

def sattel_gradient(x):
    return np.array(
        [
            x[0]**2 * (0.6 * x[0] + 3.0) + x[1]**2 * (0.6 * x[0] - 2.0) - 0.6 * x[0],
            0.6 * x[1] * (x[0]**2 + x[1]**2 - 1) - 4.0 * x[0] * x[1],
        ]
    )

def sattel_hesse(x):
    xx = x[0]
    yy = x[1]
    x2 = xx * xx
    y2 = yy * yy
    xy = xx * yy
    return np.array(
        [
            [
                6.0 * x[0] + 1.8 * x[0]**2 + 0.6 * x[1]**2 - 1.0,
                1.2 * x[0] * x[1] - 4.0 * x[1],
            ],
            [
                1.2  * x[0] * x[1] - 4.0 * x[1],
                -4.0 * x[0] + 1.8 * x[1]**2 + 0.6 * x[0]**2 - 1.0,
            ],
        ],
    )

sattel = (sattel_funktion, sattel_gradient, sattel_hesse)

Mit der Rosenbrock-Funktion können Sie nun nocheinmal Ihre Implementierung testen. Dabei sollten beide zum selben Minimum (1, 1) konvergieren, und `newton_local` ungefähr 7, `newton_global` ungefähr 22 Schritte benötigen.

In [ ]:
iterates = newton_local(*rosenbrock, np.array([-1.2, 1.0]), 1e-16, 100)
print(f'newton_local konvergiert in {len(iterates)-1} Schritten zu {iterates[-1]}.')

In [ ]:
iterates = newton_global(*rosenbrock, np.array([-1.2, 1.0]), 1e-16, 100, armijo_step_size)
print(f'newton_global konvergiert in {len(iterates)-1} Schritten zu {iterates[-1]}.')

Abschließend können wir nun das lokale und das globale Newton-Verfahren und ihre Abhängigkeit vom Startwert untersuchen. Die Plots zeigen dabei jeweils die benötigten Iterationen bis zur Konvergenz der Verfahren. Sie können gerne die Auflösung des Gitters erhöhen, dies dauert aber natürlich entsprechend länger.

In [ ]:
from util.plotting_07 import plot_07

resolution = (60, 40)

In [ ]:
x_range = (-1.5, 1.5)
y_range = (-0.4, 1.6)
plot_07(newton_local, newton_global, armijo_step_size, *rosenbrock, *resolution, *x_range, *y_range)

In [ ]:
x_range = (-7.5, 7.5)
y_range = (-5.0, 5.0)
plot_07(newton_local, newton_global, armijo_step_size, *himmelblau, *resolution, *x_range, *y_range)

In [ ]:
x_range = (-9.0, 6.0)
y_range = (-5.0, 5.0)
plot_07(newton_local, newton_global, armijo_step_size, *sattel, *resolution, *x_range, *y_range)

Besonders bei der letzten Funktion fällt auf, dass das globale Newton-Verfahren auch für Werte konvergiert, für die das lokale Newton-Verfahren in der vorgegebenen Iterationsanzahl keine Lösung liefert (was als gelbe Bereiche im Plot sichtbar ist).